In [1]:
import pandas as pd

TRAIN = '/kaggle/input/digit-recognizer/train.csv'
train_df = pd.read_csv(filepath_or_buffer=TRAIN, )
print('training data load complete.')

training data load complete.


In [2]:
from warnings import filterwarnings
from plotly import express
filterwarnings(action='ignore', category=UserWarning)

express.histogram(data_frame=train_df, x='label',).show()
express.pie(data_frame=train_df, names='label', color='label').show()

Let's just take a few iterations of UMAP and see how our training data clusters.

In [3]:
from arrow import now
from umap import UMAP

time_start = now()
columns = [column for column in train_df.columns if column.startswith('pixel')]
umap = UMAP(random_state=2024, verbose=True, n_jobs=1, low_memory=False, n_epochs=100)
train_df[['x', 'y']] = pd.DataFrame(data=umap.fit_transform(X=train_df[columns]))
express.scatter(data_frame=train_df, x='x', y='y', color='label').show()
print('done with UMAP in {}'.format(now() - time_start))

2024-03-29 15:01:14.777789: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-29 15:01:14.777995: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-29 15:01:14.944937: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


UMAP(low_memory=False, n_epochs=100, n_jobs=1, random_state=2024, verbose=True)
Fri Mar 29 15:01:28 2024 Construct fuzzy simplicial set
Fri Mar 29 15:01:28 2024 Finding Nearest Neighbors
Fri Mar 29 15:01:28 2024 Building RP forest with 15 trees
Fri Mar 29 15:01:36 2024 NN descent for 15 iterations
	 1  /  15
	 2  /  15
	 3  /  15
	 4  /  15
	 5  /  15
	Stopping threshold met -- exiting after 5 iterations
Fri Mar 29 15:02:01 2024 Finished Nearest Neighbor Search
Fri Mar 29 15:02:05 2024 Construct embedding


Epochs completed:   0%|            0/100 [00:00]

	completed  0  /  100 epochs
	completed  10  /  100 epochs
	completed  20  /  100 epochs
	completed  30  /  100 epochs
	completed  40  /  100 epochs
	completed  50  /  100 epochs
	completed  60  /  100 epochs
	completed  70  /  100 epochs
	completed  80  /  100 epochs
	completed  90  /  100 epochs
Fri Mar 29 15:02:23 2024 Finished embedding


done with UMAP in 0:00:56.429385


Four of our digits are for the most part in isolated clusters; the other six digits are in clusters that have regions of ambiguity.

Let's separate out a validation set and use logistic regression to set a baseline for our model.

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from arrow import now

time_start = now()
target = 'label'
X_train, X_test, y_train, y_test = train_test_split(train_df[columns], train_df[target], test_size=0.20, random_state=2024, stratify=train_df[target])

regression = LogisticRegression(max_iter=200, tol=1e-6, verbose=1)
regression.fit(X=X_train, y=y_train)
print('fit complete after {} iterations.'.format(regression.n_iter_[0]))
print('accuracy: {:5.4f}'.format(regression.score(X=X_test, y=y_test)))
print('done in {}'.format(now() - time_start))

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =         7850     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  7.73669D+04    |proj g|=  5.40622D+05


 This problem is unconstrained.



At iterate   50    f=  8.21115D+03    |proj g|=  8.66931D+03

At iterate  100    f=  7.16515D+03    |proj g|=  6.54177D+03

At iterate  150    f=  6.67735D+03    |proj g|=  5.21895D+03

At iterate  200    f=  6.33388D+03    |proj g|=  5.11963D+03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
 7850    200    219      1     0     0   5.120D+03   6.334D+03
  F =   6333.8759625519724     

STOP: TOTAL NO. of ITERATIONS REACHED LIMIT                 
fit complete after 200 iterations.
accuracy: 0.9107
done in 0:00:17.597920


In [5]:
from arrow import now
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

MODEL = {
    'Neural Net 1.0': MLPClassifier(alpha=1, max_iter=2000, random_state=2024),
}

# we have tried k in {3, 5} here; 3 is best so far
for n_neighbors in range(2, 12):
    MODEL['{} Nearest Neighbors'.format(n_neighbors)] = KNeighborsClassifier(n_neighbors=n_neighbors)

for n_estimators in range(5, 100, 5):
    MODEL['{} estimator Random Forest'.format(n_estimators)] = RandomForestClassifier(n_estimators=n_estimators,  random_state=2024)

result = []
for name, clf in MODEL.items():
    time_start = now()
    clf.fit(X=X_train, y=y_train)
    score = clf.score(X=X_test, y=y_test)
    result.append((score, name))
    print('{:5.4f} {} {}'.format(score, now() - time_start, name))
result = sorted(result, key=lambda x: x[0], reverse=True)
print('best: {:5.4f} {}'.format(result[0][0], result[0][1]))

0.9613 0:01:53.251747 Neural Net 1.0
0.9601 0:00:10.770154 2 Nearest Neighbors
0.9667 0:00:11.475931 3 Nearest Neighbors
0.9661 0:00:12.078835 4 Nearest Neighbors
0.9661 0:00:11.803509 5 Nearest Neighbors
0.9650 0:00:12.187556 6 Nearest Neighbors
0.9642 0:00:11.742062 7 Nearest Neighbors
0.9632 0:00:11.794972 8 Nearest Neighbors
0.9626 0:00:11.939283 9 Nearest Neighbors
0.9619 0:00:11.812640 10 Nearest Neighbors
0.9613 0:00:11.831964 11 Nearest Neighbors
0.9079 0:00:01.481620 5 estimator Random Forest
0.9385 0:00:02.768186 10 estimator Random Forest
0.9476 0:00:04.041891 15 estimator Random Forest
0.9532 0:00:05.415574 20 estimator Random Forest
0.9564 0:00:06.819483 25 estimator Random Forest
0.9568 0:00:08.091276 30 estimator Random Forest
0.9577 0:00:09.512354 35 estimator Random Forest
0.9599 0:00:10.754073 40 estimator Random Forest
0.9600 0:00:11.980561 45 estimator Random Forest
0.9610 0:00:13.335250 50 estimator Random Forest
0.9614 0:00:14.783114 55 estimator Random Forest
0.9

Let's make our submission.

This is our best submission so far; it's cheap to generate so let's generate it every time.

In [6]:
import arrow

TEST = '/kaggle/input/digit-recognizer/test.csv'

time_start = arrow.now()
best = KNeighborsClassifier(n_neighbors=3)
best.fit(X=train_df[columns], y=train_df[target])

best_df = pd.DataFrame(data=best.predict(X=pd.read_csv(filepath_or_buffer=TEST, ))).reset_index()
best_df.columns = ['ImageId', 'Label']
best_df['ImageId'] = best_df['ImageId'] + 1
best_file = '/kaggle/working/best.csv.zip'
print('{} : writing best so far guess to {}'.format(arrow.now() - time_start, best_file))
best_df.to_csv(path_or_buf=best_file, index=False, compression='zip')
print('{} : done.'.format(arrow.now() - time_start, ))

0:00:50.944282 : writing best so far guess to /kaggle/working/best.csv.zip
0:00:51.000710 : done.


In [7]:
import arrow

TEST = '/kaggle/input/digit-recognizer/test.csv'

time_start = arrow.now()
current = RandomForestClassifier(n_estimators=45,  random_state=2024).fit(X=train_df[columns], y=train_df[target])

result_df = pd.DataFrame(data=current.predict(X=pd.read_csv(filepath_or_buffer=TEST, ))).reset_index()
result_df.columns = ['ImageId', 'Label']
result_df['ImageId'] = result_df['ImageId'] + 1
output_file = '/kaggle/working/submission.csv.zip'
print('{} : writing submission to {}'.format(arrow.now() - time_start, output_file))
result_df.to_csv(path_or_buf=output_file, index=False, compression='zip')
print('{} : done.'.format(arrow.now() - time_start, ))

0:00:18.271799 : writing submission to /kaggle/working/submission.csv.zip
0:00:18.327231 : done.


In [8]:
test_df = pd.read_csv(filepath_or_buffer=TEST, )
test_df[['x', 'y']] = pd.DataFrame(data=umap.fit_transform(X=test_df))
test_df['Label'] = result_df['Label'].tolist()
express.scatter(data_frame=test_df, x='x', y='y', color='Label').show()

UMAP(low_memory=False, n_epochs=100, n_jobs=1, random_state=2024, verbose=True)
Fri Mar 29 15:11:59 2024 Construct fuzzy simplicial set
Fri Mar 29 15:11:59 2024 Finding Nearest Neighbors
Fri Mar 29 15:11:59 2024 Building RP forest with 13 trees
Fri Mar 29 15:12:01 2024 NN descent for 15 iterations
	 1  /  15
	 2  /  15
	 3  /  15
	 4  /  15
	 5  /  15
	Stopping threshold met -- exiting after 5 iterations
Fri Mar 29 15:12:06 2024 Finished Nearest Neighbor Search
Fri Mar 29 15:12:06 2024 Construct embedding


Epochs completed:   0%|            0/100 [00:00]

	completed  0  /  100 epochs
	completed  10  /  100 epochs
	completed  20  /  100 epochs
	completed  30  /  100 epochs
	completed  40  /  100 epochs
	completed  50  /  100 epochs
	completed  60  /  100 epochs
	completed  70  /  100 epochs
	completed  80  /  100 epochs
	completed  90  /  100 epochs
Fri Mar 29 15:12:17 2024 Finished embedding


In [9]:
import arrow
from sklearn.ensemble import  VotingClassifier

time_start = arrow.now()
voting = VotingClassifier(estimators=[('rs: {}'.format(random_state), RandomForestClassifier(n_estimators=25, random_state=random_state)) for random_state in range(1000, 1011)], voting='hard').fit(X=train_df[columns], y=train_df[target])
print('{} : built/trained model.'.format(arrow.now() - time_start, ))
voting_df = pd.DataFrame(columns=['Label'], data=current.predict(X=test_df[columns]))
voting_df['ImageId'] = list(range(1, len(voting_df)+1))
voting_file = '/kaggle/working/voting.csv.zip'
print('{} : writing submission to {}'.format(arrow.now() - time_start, voting_file))
voting_df.to_csv(path_or_buf=voting_file, index=False, compression='zip')
print('{} : done.'.format(arrow.now() - time_start, ))

0:01:42.800516 : built/trained model.
0:01:43.428897 : writing submission to /kaggle/working/voting.csv.zip
0:01:43.486405 : done.
